In [1]:
!pip install ultralytics
!pip install -U ipywidgets

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/6b/f6/5d5ffef34686a1e99f819a2cf91db7186def16c5b020fb6e1cb7766e968d/ultralytics-8.1.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 743.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 701.1/701.1 kB 5.9 MB/s eta 0:00:00
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0

In [2]:
!mkdir -p /kaggle/temp/train/labels
!mkdir -p /kaggle/temp/train/images

!mkdir -p /kaggle/temp/val/labels
!mkdir -p /kaggle/temp/val/images

In [3]:
!cp /kaggle/input/kidney-1-dense-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-dense-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-1-voi-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-1-voi-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-2-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-2-to-yolo/train/images/* /kaggle/temp/train/images

!cp /kaggle/input/kidney-3-sparse-to-yolo/train/labels/* /kaggle/temp/train/labels
!cp /kaggle/input/kidney-3-sparse-to-yolo/train/images/* /kaggle/temp/train/images

In [4]:
!cp /kaggle/input/kidney-3-dense-to-yolo/train/labels/* /kaggle/temp/val/labels
!cp /kaggle/input/kidney-3-dense-to-yolo/train/images/* /kaggle/temp/val/images

In [5]:
# !tail /kaggle/input/kidney-1-voi-to-yolo/train/kidney_1_voi.json

# Write yolo dataset yaml

In [6]:
import yaml

In [7]:
!mkdir train

In [8]:
train_path = '/kaggle/temp/train'
val_path = '/kaggle/temp/val'
names = {0: 'Vessel'}
yaml_path = 'train/train.yaml'

# Number of classes
nc = len(names)

# Create a dictionary with the required content
yaml_data = {
    'names': names,
    'nc': nc,
    'test': '',
    'train': train_path,
    'val': val_path
}

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

In [9]:
!cat {yaml_path}

names:
  0: Vessel
nc: 1
test: ''
train: /kaggle/temp/train
val: /kaggle/temp/val


In [10]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
from ultralytics import YOLO

model = YOLO('yolov8l-seg.pt')

100%|██████████| 88.1M/88.1M [00:00<00:00, 249MB/s]


In [12]:
project = 'hacking_human_vasculature'
name = 'kidney_3_dense_val_rest_train_yolo_xl_img_size_1952'

results = model.train(data=yaml_path,
                      project=project,
                      name=name,
                      epochs=20,
                      time=5,
                      patience=5, 
                      batch=2,
                      imgsz=1952,
                      device=[0, 1],
                      save_period=1,
                      cache=True
                     )

Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8l-seg.pt, data=train/train.yaml, epochs=20, time=5, patience=5, batch=2, imgsz=1952, save=True, save_period=1, cache=True, device=[0, 1], workers=8, project=hacking_human_vasculature, name=kidney_3_dense_val_rest_train_yolo_xl_img_size_1952, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

100%|██████████| 755k/755k [00:00<00:00, 15.0MB/s]
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


TensorBoard: Start with 'tensorboard --logdir hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_xl_img_size_1952', view at http://localhost:6006/


...

        8/8        12G     0.7745     0.8085     0.4396     0.8067         25       1952:  67%|██████▋   | 2332/3464 [24:00<11:39,  1.62it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 518/518 [03:21<00:00,  2.58it/s]


                   all       1035      41540      0.532       0.48      0.378      0.219       0.42      0.401      0.321      0.151
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 3, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

8 epochs completed in 5.060 hours.
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_xl_img_size_1952/weights/last.pt, 92.7MB
Optimizer stripped from hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_xl_img_size_1952/weights/best.pt, 92.7MB

Validating hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_xl_img_size_1952/weights/best.pt...
Ultralytics YOLOv8.1.3 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla T4, 15102MiB)
                                                     CUDA:1 (Tesla T4, 15102MiB)
YOLOv8l-seg summary (fused): 295 layers, 4591265

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 518/518 [03:08<00:00,  2.74it/s]


                   all       1035      41540      0.455      0.466      0.395      0.247      0.391      0.399       0.33      0.173
Speed: 1.8ms preprocess, 165.2ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to hacking_human_vasculature/kidney_3_dense_val_rest_train_yolo_xl_img_size_1952


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆█▇▅▄▂▁
wandb:                  lr/pg1 ▃▆█▇▅▄▂▁
wandb:                  lr/pg2 ▃▆█▇▅▄▂▁
wandb:        metrics/mAP50(B) ▃▁█▃▆▆▆█
wandb:        metrics/mAP50(M) ▁▂▅▄█▇▄▅
wandb:     metrics/mAP50-95(B) ▁▁█▂▃▃▃█
wandb:     metrics/mAP50-95(M) ▁▃█▅▇▇▃█
wandb:    metrics/precision(B) ▇▆▁▇█▇█▁
wandb:    metrics/precision(M) ▅▆▁▆█▅▆▁
wandb:       metrics/recall(B) ▄▁▆▆▅█▅▆
wandb:       metrics/recall(M) ▁▁▄▄▄█▃▄
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:          train/box_loss █▅▅▄▃▂▂▁
wandb:          train/cls_loss █▃▃▂▂▁▁▁
wandb:          train/dfl_loss █▅▄▄▃▃▂▁
wandb:          train/seg_loss █▄▄▃▃▂▂▁
wandb:            val/box_loss █▄▄▁▇▃▆▁
wandb:            val/cls_loss ▅▅▁▆▅█▇█
wandb:            val/dfl_loss ▆▆█▅▆▄▆▁
wandb:            val/seg_loss ▄▅▁▂▇▅█▁
wandb: 
wandb: Run summary

In [13]:
wandb.finish()